In [ ]:
# from google.colab import files
# files.upload()  # upload your kaggle.json file

# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json


# ! kaggle datasets download -d vinayaks0n1/imdb-tv-show-reviews

# import zipfile
# with zipfile.ZipFile("imdb-tv-show-reviews.zip","r") as zip_ref:
#   zip_ref.extractall("dataset")

# dataset="dataset"

import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import re

df=pd.read_csv(r"C:\Users\Arun\Downloads\imdb-tv-show-reviews\imdb_tvshows.csv")

df.head()

def clean(text):
  text=str(text).lower()
  text=re.sub(r"[^a-zA-Z\s]","",text)
  return text
df["Review"]=df["Review"].apply(clean)

data=" ".join(df["Review"].to_list())

token=Tokenizer()
token.fit_on_texts([data])
total_words=len(token.word_index)+1

tokens=token.texts_to_sequences([data])[0]
tokens

from collections.abc import Sequence
input_sequence=[]
max_length=30
for i in range(1,total_words):
  Sequence=tokens[max(0,i-max_length):i+1]
  input_sequence.append(Sequence)


total_words

lenght=max([len(seq) for seq in input_sequence])

from tensorflow.keras.preprocessing.sequence import pad_sequences

pad_sequence=pad_sequences(input_sequence,maxlen=lenght,padding="pre")

pad_sequence

x=pad_sequence[:,:-1]
y=pad_sequence[:,-1]

x

y

from tensorflow.keras.utils import to_categorical

y=to_categorical(y,num_classes=total_words)
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding,Dense,LSTM,Dropout
from tensorflow.keras.callbacks import EarlyStopping

x.shape
with tf.device("/cpu:0"):
    model=Sequential()
    model.add(Embedding(input_dim=total_words,output_dim=64))
    model.add(LSTM(100))
    model.add(Dense(total_words,activation="softmax"))
  

with tf.device("/cpu:0"):
    model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
    model.build(input_shape=(None,x.shape[1]))
    model.summary()

stopper=EarlyStopping(
    patience=3,
    monitor="val_accuracy",
    verbose=1,
    restore_best_weights=True
)
with tf.device("/cpu:0"):
    hist=model.fit(x,y,epochs=100,callbacks=stopper,validation_split=0.3)



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          1779200   
                                                                 
 lstm_2 (LSTM)               (None, 100)               66000     
                                                                 
 dense_2 (Dense)             (None, 27800)             2807800   
                                                                 
Total params: 4,653,000
Trainable params: 4,653,000
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
609/609 [==============================] - 26s 41ms/step - loss: 7.0145 - accuracy: 0.0610 - val_loss: 6.7846 - val_accuracy: 0.0616
Epoch 2/100
609/609 [==============================] - 23s 38ms/step - loss: 6.2428 - accuracy: 0.0677 - val_loss: 6.7962 - val_accuracy: 0.0748
Epoch 3/100
495/60